In [1]:
import logging
import os
import random
import sys
from dataclasses import dataclass, field
from typing import Optional

import datasets
import numpy as np
from datasets import load_dataset

import torch
import torch.nn.functional as F

import evaluate
import transformers
from transformers import (
    AutoConfig,
    AutoModelForSequenceClassification,
    AutoTokenizer,
    DataCollatorWithPadding,
    EvalPrediction,
    HfArgumentParser,
    PretrainedConfig,
    Trainer,
    TrainingArguments,
    default_data_collator,
    set_seed,
)
from transformers.trainer_utils import get_last_checkpoint
from transformers.utils import check_min_version, send_example_telemetry
from transformers.utils.versions import require_version

import matplotlib.pyplot as plt

comet_ml is installed but `COMET_API_KEY` is not set.


In [2]:
import nltk

# 1. check the range of discrete perturbation

In [3]:
def get_avg_disc_cos_sim_1212(textfooler_tensor, timestep_0, timestep_1):
    num_disc_examples = 0
    layer_avg_output_disc = {}

    for layer_id in range(1, 13):
        layer_key = 'layer_' + str(layer_id)
        layer_avg_output_disc[layer_key] = torch.tensor(0.0)


    for textfooler_t0 in textfooler_tensor:
        if not textfooler_t0['timestep'] == timestep_0:
            continue

        textfooler_t1 = None

        for item in textfooler_tensor:
            # 找到timestep_1的对应text_attack样本
            if textfooler_t0['input_ids'].equal(item['input_ids']) and item['timestep'] == timestep_1:
                textfooler_t1 = item
                continue

        # 保证找到结果（离散空间不一定找得到）
        if textfooler_t1:
            num_disc_examples += 1
        else:
            continue

        # 6. 计算离散空间的[CLS]表示，以及计算连续和离散的相似度
        for layer_id in range(1, 13):
            layer_key = 'layer_' + str(layer_id)
            # \cos( f(x_{t+1}), f(x_t) )
            disc_sim = F.cosine_similarity(textfooler_t1[layer_key], textfooler_t0[layer_key], dim=0)
            # if layer_id == 12:
            #     print(disc_sim)
            #     print(textfooler_t1['raw_input'])
            #     print(textfooler_t1['attacked_input'])
            #     mynum += 1
            #     print()
            # \cos( cont_sim, disc_sim )
            layer_avg_output_disc[layer_key] += disc_sim

        # averge on num_examples
    for layer_id in range(1, 13):
        layer_key = 'layer_' + str(layer_id)
        layer_avg_output_disc[layer_key] = (layer_avg_output_disc[layer_key] / num_disc_examples).item()

    print("num_disc_examples: {}".format(num_disc_examples))

    return layer_avg_output_disc

In [4]:
# a. imdb-bert-textfooler
textfooler_imdb_tensor = torch.load('/remote-home/kren/exps/TextAttack/tensors/textattack_bert_imdb_1000sample_20221211.pt')

# b. imdb-bert-random
# textfooler_imdb_tensor = torch.load('/remote-home/kren/exps/TextAttack/tensors/textfooler_imdb_bert_random_1000sample_20221226.pt')

print(len(textfooler_imdb_tensor))

10866


In [5]:
timestep_0 = 0
for timestep_1 in [1, 3, 5, 8, 10, 13, 15, 20, 25, 30]:
    layer_avg_output_disc_1212 = get_avg_disc_cos_sim_1212(textfooler_imdb_tensor, timestep_0=timestep_0, timestep_1=timestep_1)
    print("timestep_0: {}, timestep_1: {}, cos(s,s')={}".format
          (timestep_0, timestep_1, round(layer_avg_output_disc_1212['layer_12'], 4)))

num_disc_examples: 999
timestep_0: 0, timestep_1: 1, cos(s,s')=0.9022
num_disc_examples: 999
timestep_0: 0, timestep_1: 3, cos(s,s')=0.7312
num_disc_examples: 999
timestep_0: 0, timestep_1: 5, cos(s,s')=0.601
num_disc_examples: 998
timestep_0: 0, timestep_1: 8, cos(s,s')=0.4706
num_disc_examples: 998
timestep_0: 0, timestep_1: 10, cos(s,s')=0.4093
num_disc_examples: 998
timestep_0: 0, timestep_1: 13, cos(s,s')=0.3352
num_disc_examples: 996
timestep_0: 0, timestep_1: 15, cos(s,s')=0.2965
num_disc_examples: 981
timestep_0: 0, timestep_1: 20, cos(s,s')=0.2244
num_disc_examples: 961
timestep_0: 0, timestep_1: 25, cos(s,s')=0.1713
num_disc_examples: 938
timestep_0: 0, timestep_1: 30, cos(s,s')=0.1247


# 2. 构造样本对-IMDB

In [6]:
def get_cont_cos_sim_1213(freelb_tensor, timestep_0=0, timestep_1=1):
    """
    在连续的情况下，一定是
        timestep_0 = 0
        timestep_1 = 1
    return: dict,
        key = input_id
        value = 12th layer [cls] cos_sim
    """
    # 新建一个dict，key是input_id，value是最后一层的cos_sim
    cont_cos_dict = {}

    for freelb_t0 in freelb_tensor:
        if not freelb_t0['timestep'] == timestep_0:
            continue

        freelb_t1 = None

        for item in freelb_tensor:
            # 找到timestep_1的对应text_attack样本
            if freelb_t0['input_ids'].equal(item['input_ids']) and item['timestep'] == timestep_1:
                freelb_t1 = item
                continue

        # 保证找到结果（连续空间一定能找到，离散空间不一定找得到）
        if not freelb_t1:
            continue

        key = str(freelb_t0['input_ids'])
        value = round(F.cosine_similarity(freelb_t0['layer_12'], freelb_t1['layer_12'], dim=0).item(), 4)

        cont_cos_dict[key] = value

    print("cont_cos_dict.len = {}".format(len(cont_cos_dict)))
    return cont_cos_dict


def get_disc_cos_sim_1213(textfooler_tensor, timestep_0, timestep_1):
    """
    return: d
    disc_cos_dict: key=input_id, value=[cls] cos sim，用于对比s和x
    diff_t_dict: key=input_id, value=textfooler_t1，用于读取s和s'
    """
    disc_cos_dict = {}
    diff_t_dict = {}

    for textfooler_t0 in textfooler_tensor:
        if not textfooler_t0['timestep'] == timestep_0:
            continue

        textfooler_t1 = None

        for item in textfooler_tensor:
            # 找到timestep_1的对应text_attack样本
            if textfooler_t0['input_ids'].equal(item['input_ids']) and item['timestep'] == timestep_1:
                textfooler_t1 = item
                continue

        # 保证找到结果（离散空间不一定找得到）
        if not textfooler_t1:
            continue

        key = str(textfooler_t0['input_ids'])
        value = round(F.cosine_similarity(textfooler_t0['layer_12'], textfooler_t1['layer_12'], dim=0).item(), 4)

        disc_cos_dict[key] = value
        diff_t_dict[key] = textfooler_t1

    print("timestep_1 = {}, disc_cos_dict.len = {}".format(timestep_1, len(disc_cos_dict)))
    return disc_cos_dict, diff_t_dict

In [7]:
# 1. 根据不同的时间步t，加载离散的数据集，并存到字典all_disc_cos_dict中，key是timestep_1，value是对应的input_id和cos_sim的dict
# all_diff_t_dict: key=input_id, value=textfooler_t1，用于读取s和s'


# a. imdb-bert-textfooler
textfooler_tensor = torch.load('/remote-home/kren/exps/TextAttack/tensors/textattack_bert_imdb_1000sample_20221211.pt')

# b. imdb-bert-random
# textfooler_tensor = torch.load('/remote-home/kren/exps/TextAttack/tensors/textfooler_imdb_bert_random_1000sample_20221226.pt')

print("len(textfooler_tensor) = {}".format(len(textfooler_tensor)))

len(textfooler_tensor) = 10866


In [8]:
# timestep_1_list = [1, 3, 5]
# 985 963 945
timestep_1_list = [1, 3, 5, 8, 10, 13, 15, 20, 25, 30]
all_disc_cos_dict = {}
all_diff_t_dict = {}
for timestep_1 in timestep_1_list:
    key = timestep_1
    disc_cos_dict, diff_t_dict = get_disc_cos_sim_1213(textfooler_tensor, timestep_0=0, timestep_1=timestep_1)
    all_disc_cos_dict[key] = disc_cos_dict
    all_diff_t_dict[key] = diff_t_dict

timestep_1 = 1, disc_cos_dict.len = 999
timestep_1 = 3, disc_cos_dict.len = 999
timestep_1 = 5, disc_cos_dict.len = 999
timestep_1 = 8, disc_cos_dict.len = 998
timestep_1 = 10, disc_cos_dict.len = 998
timestep_1 = 13, disc_cos_dict.len = 998
timestep_1 = 15, disc_cos_dict.len = 996
timestep_1 = 20, disc_cos_dict.len = 981
timestep_1 = 25, disc_cos_dict.len = 961
timestep_1 = 30, disc_cos_dict.len = 938


In [9]:
# 2. 根据不同的anorm，加载连续的数据集，并存到字典all_cont_cos_dict中，key是anorm，value是对应的input_id和cos_sim的dict
adv_max_norm_list = []

# [0.5e-1, 0.9e-1]
for i in range(0, 1):
    for j in range(5, 10):
        print("{}.{}e-1".format(i, j))
        adv_max_norm_list.append("{}.{}e-1".format(i, j))

# [1e-1, 8e-1)
for i in range(1, 8):
    print("{}e-1".format(i))
    adv_max_norm_list.append("{}e-1".format(i))
    for j in range(1, 10):
        print("{}.{}e-1".format(i,j))
        adv_max_norm_list.append("{}.{}e-1".format(i,j))

# [8e-1]
for i in range(8, 9):
    print("{}e-1".format(i))
    adv_max_norm_list.append("{}e-1".format(i))
adv_max_norm_list

0.5e-1
0.6e-1
0.7e-1
0.8e-1
0.9e-1
1e-1
1.1e-1
1.2e-1
1.3e-1
1.4e-1
1.5e-1
1.6e-1
1.7e-1
1.8e-1
1.9e-1
2e-1
2.1e-1
2.2e-1
2.3e-1
2.4e-1
2.5e-1
2.6e-1
2.7e-1
2.8e-1
2.9e-1
3e-1
3.1e-1
3.2e-1
3.3e-1
3.4e-1
3.5e-1
3.6e-1
3.7e-1
3.8e-1
3.9e-1
4e-1
4.1e-1
4.2e-1
4.3e-1
4.4e-1
4.5e-1
4.6e-1
4.7e-1
4.8e-1
4.9e-1
5e-1
5.1e-1
5.2e-1
5.3e-1
5.4e-1
5.5e-1
5.6e-1
5.7e-1
5.8e-1
5.9e-1
6e-1
6.1e-1
6.2e-1
6.3e-1
6.4e-1
6.5e-1
6.6e-1
6.7e-1
6.8e-1
6.9e-1
7e-1
7.1e-1
7.2e-1
7.3e-1
7.4e-1
7.5e-1
7.6e-1
7.7e-1
7.8e-1
7.9e-1
8e-1


['0.5e-1',
 '0.6e-1',
 '0.7e-1',
 '0.8e-1',
 '0.9e-1',
 '1e-1',
 '1.1e-1',
 '1.2e-1',
 '1.3e-1',
 '1.4e-1',
 '1.5e-1',
 '1.6e-1',
 '1.7e-1',
 '1.8e-1',
 '1.9e-1',
 '2e-1',
 '2.1e-1',
 '2.2e-1',
 '2.3e-1',
 '2.4e-1',
 '2.5e-1',
 '2.6e-1',
 '2.7e-1',
 '2.8e-1',
 '2.9e-1',
 '3e-1',
 '3.1e-1',
 '3.2e-1',
 '3.3e-1',
 '3.4e-1',
 '3.5e-1',
 '3.6e-1',
 '3.7e-1',
 '3.8e-1',
 '3.9e-1',
 '4e-1',
 '4.1e-1',
 '4.2e-1',
 '4.3e-1',
 '4.4e-1',
 '4.5e-1',
 '4.6e-1',
 '4.7e-1',
 '4.8e-1',
 '4.9e-1',
 '5e-1',
 '5.1e-1',
 '5.2e-1',
 '5.3e-1',
 '5.4e-1',
 '5.5e-1',
 '5.6e-1',
 '5.7e-1',
 '5.8e-1',
 '5.9e-1',
 '6e-1',
 '6.1e-1',
 '6.2e-1',
 '6.3e-1',
 '6.4e-1',
 '6.5e-1',
 '6.6e-1',
 '6.7e-1',
 '6.8e-1',
 '6.9e-1',
 '7e-1',
 '7.1e-1',
 '7.2e-1',
 '7.3e-1',
 '7.4e-1',
 '7.5e-1',
 '7.6e-1',
 '7.7e-1',
 '7.8e-1',
 '7.9e-1',
 '8e-1']

In [ ]:
# adv_max_norm_list = ['1e-1', '1.1e-1']
all_cont_cos_dict = {}

for anorm in adv_max_norm_list:
    adv_lr = '1e-1'
    adv_init_mag = '0e-0'
    adv_max_norm = anorm
    adv_steps = '15'
    root_path = '/remote-home/kren/exps/FreeLB/hf424_torch182/tensors/'
    file_name = 'freelb_bert_imdb_alr' + str(adv_lr) + '_amag' + str(adv_init_mag) + '_anm' + str(adv_max_norm) + '_as' + str(adv_steps)
    freelb_path = root_path + file_name + '.pt'
    print(freelb_path)
    freelb_tensor = torch.load(freelb_path)

    key = anorm
    value = get_cont_cos_sim_1213(freelb_tensor, timestep_0=0, timestep_1=1)
    all_cont_cos_dict[key] = value

/remote-home/kren/exps/FreeLB/hf424_torch182/tensors/freelb_bert_imdb_alr1e-1_amag0e-0_anm0.5e-1_as15.pt
cont_cos_dict.len = 1000
/remote-home/kren/exps/FreeLB/hf424_torch182/tensors/freelb_bert_imdb_alr1e-1_amag0e-0_anm0.6e-1_as15.pt
cont_cos_dict.len = 1000
/remote-home/kren/exps/FreeLB/hf424_torch182/tensors/freelb_bert_imdb_alr1e-1_amag0e-0_anm0.7e-1_as15.pt
cont_cos_dict.len = 1000
/remote-home/kren/exps/FreeLB/hf424_torch182/tensors/freelb_bert_imdb_alr1e-1_amag0e-0_anm0.8e-1_as15.pt
cont_cos_dict.len = 1000
/remote-home/kren/exps/FreeLB/hf424_torch182/tensors/freelb_bert_imdb_alr1e-1_amag0e-0_anm0.9e-1_as15.pt
cont_cos_dict.len = 1000
/remote-home/kren/exps/FreeLB/hf424_torch182/tensors/freelb_bert_imdb_alr1e-1_amag0e-0_anm1e-1_as15.pt
cont_cos_dict.len = 1000
/remote-home/kren/exps/FreeLB/hf424_torch182/tensors/freelb_bert_imdb_alr1e-1_amag0e-0_anm1.1e-1_as15.pt
cont_cos_dict.len = 1000
/remote-home/kren/exps/FreeLB/hf424_torch182/tensors/freelb_bert_imdb_alr1e-1_amag0e-0_anm1.

In [303]:
# 3. 开始搜索，all_examples存放所有搜索到的结果
all_examples = []

# a. 遍历不同时间步下离散的扰动
for timestep, disc_dict in all_disc_cos_dict.items():

    # b. 遍历单个时间步下离散的扰动，no_find_num统计没有找到相似的数量, range_list用于保存差值范围较大的结果
    print("searching timestep={} disc perturbation".format(timestep))
    no_find_num = 0
    for disc_input, disc_sim in disc_dict.items():
        find_flag = False
        range1_list = []
        range2_list = []

        # c. 遍历连续的扰动，寻找与离散的扰动最接近的那一个
        for epsilon, cont_dict in all_cont_cos_dict.items():
            # d. 找到对应epsilon的连续扰动的cos_sim (cont_sim)
            try:
                cont_sim = cont_dict[disc_input]
            except:
                print("wrong! check code!")
                continue

            one_example = {"raw": all_diff_t_dict[timestep][disc_input]['raw_input'],
                           "attacked": all_diff_t_dict[timestep][disc_input]['attacked_input'],
                           "epsilon": float(epsilon),
                           "timestep": timestep,
                           "cont_sim":cont_sim,
                           "disc_sim": disc_sim,
                           "difference": ""}

            # if disc_sim <= 0.0:
            #     # 如果cos(s, s') <= 0，则按0计算，此时对应连续扰动的epsilon=6e-1
            #     find_flag = True
            #     one_example['epsilon'] = 6e-1
            #     one_example['difference'] = "cos<0"
            #     all_examples.append(one_example)
            #     break
            #
            # if disc_sim <= 0.0:
            #     # 如果cos(s, s') <= 0，则按0.0考虑
            #     disc_sim = 0.0

            if abs(cont_sim - disc_sim) <= 0.005:
                # 如果差值 <= 0.005，说明成功找到了
                find_flag = True
                one_example['difference'] = '0.005'
                all_examples.append(one_example)
                break
            elif abs(cont_sim - disc_sim) <= 0.01:
                # 如果没找到差值 <= 0.005，则开始找差值 <= 0.01
                one_example['difference'] = '0.01'
                range1_list.append(one_example)
            elif abs(cont_sim - disc_sim) <= 0.02:
                # 如果没找到差值 <= 0.01，则开始找差值 <= 0.02
                one_example['difference'] = '0.02'
                range2_list.append(one_example)

        # 没有找到相差<=0.005的结果
        if not find_flag:
            if range1_list:
                all_examples.append(range1_list[0])
            elif range2_list:
                all_examples.append(range2_list[0])
            else:
                no_find_num += 1

    print("timestep={}, no result num={}".format(timestep, no_find_num))

print("searing done")

searching timestep=1 disc perturbation
timestep=1, no result num=107
searching timestep=3 disc perturbation
timestep=3, no result num=97
searching timestep=5 disc perturbation
timestep=5, no result num=85
searching timestep=8 disc perturbation
timestep=8, no result num=80
searching timestep=10 disc perturbation
timestep=10, no result num=73
searching timestep=13 disc perturbation
timestep=13, no result num=83
searching timestep=15 disc perturbation
timestep=15, no result num=88
searching timestep=20 disc perturbation
timestep=20, no result num=116
searching timestep=25 disc perturbation
timestep=25, no result num=115
searching timestep=30 disc perturbation
timestep=30, no result num=135
searing done


# 3. 构造meta_scorer输入

1. 循环遍历raw_sentence和attacked_sentence
2. 当出现不一致时：
    1. 从这个不一致的位置想办法提取出一个词（分词）
    2. 构建模型的输入
    3. 继续循环遍历

## a. 根据raw和attacked构建输入

In [199]:
from nltk.tokenize import word_tokenize

def get_changed_text(s_raw, s_att):
    """
    s_raw: 原始句子
    s_att: 攻击后的句子
    return：word1[word2]的格式
    """
    tokenized_raw = word_tokenize(s_raw)
    tokenized_attacked = word_tokenize(s_att)
    tidx = 0
    sidx = 0
    new_input = ''
    while sidx < len(s_raw):
        if s_raw[sidx] == ' ':
            new_input += ' '
            sidx += 1
        elif (s_raw[sidx] == '"' and tokenized_raw[tidx] == "``") or (s_raw[sidx] == '"' and tokenized_raw[tidx] == "''"):
            # word_tokenize会将双引号分词为两个反引号
            # https://stackoverflow.com/questions/32185072/nltk-word-tokenize-behaviour-for-double-quotation-marks-is-confusing
            new_input += '"'
            sidx += 1
            tidx += 1
        elif (s_raw[sidx:sidx+2] == "''" and tokenized_raw[tidx] == "``") or (s_raw[sidx:sidx+2] == "''" and tokenized_raw[tidx] == "''"):
            new_input += "''"
            sidx += 2
            tidx += 1
        elif s_raw[sidx] == '\x85' or s_raw[sidx] == '\xa0' or s_raw[sidx] == '\t':
            new_input += '"'
            sidx += 1
        elif tokenized_raw[tidx] == tokenized_attacked[tidx]:
            # 如果这两个词是一样的
            # 保证结果正确
            # print(sidx)
            assert not s_raw[sidx] == ' '
            if s_raw[sidx] != tokenized_raw[tidx][0]:
                print("diff. idx: {}".format(sidx))
            assert s_raw[sidx] == tokenized_raw[tidx][0]
            new_input += tokenized_raw[tidx]
            sidx += len(tokenized_raw[tidx])
            tidx += 1
        elif tokenized_raw[tidx] != tokenized_attacked[tidx]:
            # 如果两个词不一样，则构造输入 word1[word2]
            # 保证结果正确
            assert not s_raw[sidx] == ' '
            assert s_raw[sidx] == tokenized_raw[tidx][0]
            new_input += tokenized_raw[tidx] + '[' + tokenized_attacked[tidx] + ']'
            sidx += len(tokenized_raw[tidx])
            tidx += 1
        else:
            raise Exception('wrong')
    return new_input

In [304]:
"""
one_example = {"raw": all_diff_t_dict[timestep][disc_input]['raw_input'],
               "attacked": all_diff_t_dict[timestep][disc_input]['attacked_input'],
               "epsilon": float(epsilon),
               "cont_sim":cont_sim,
               "disc_sim": disc_sim,
               "difference": ""}
"""
len(all_examples)

8982

In [305]:
from tqdm import tqdm

failed_num = 0
failed_index = []
for i in tqdm(range(len(all_examples))):
    res = None
    try:
        res = get_changed_text(all_examples[i]['raw'], all_examples[i]['attacked'])
    except:
        failed_index.append(i)
        res = ''
        failed_num += 1
    finally:
        all_examples[i]['model_input'] = res
print("total failed num: {}".format(failed_num))
print("failed index list: {}".format(failed_index))

100%|██████████████████████████████████████████████████████████████████████████████████████| 8982/8982 [00:39<00:00, 229.53it/s]

total failed num: 70
failed index list: [184, 1082, 1096, 1987, 2002, 2904, 2919, 3162, 3827, 3842, 4083, 4086, 4222, 4545, 4754, 4769, 4887, 5012, 5015, 5096, 5144, 5221, 5460, 5670, 5685, 5803, 5929, 5932, 6014, 6064, 6142, 6371, 6573, 6578, 6579, 6593, 6709, 6914, 6961, 7033, 7252, 7387, 7447, 7454, 7459, 7460, 7475, 7586, 7711, 7792, 7841, 7911, 8117, 8125, 8264, 8322, 8332, 8333, 8347, 8462, 8479, 8545, 8576, 8660, 8684, 8703, 8752, 8772, 8960, 8968]


## b. 清除失败的index

In [306]:
# 保证是倒序
failed_index = sorted(failed_index, reverse=True)
print("len(all_examples): {}".format(len(all_examples)))
print("reversed failed index list: {}".format(failed_index))

len(all_examples): 8982
reversed failed index list: [8968, 8960, 8772, 8752, 8703, 8684, 8660, 8576, 8545, 8479, 8462, 8347, 8333, 8332, 8322, 8264, 8125, 8117, 7911, 7841, 7792, 7711, 7586, 7475, 7460, 7459, 7454, 7447, 7387, 7252, 7033, 6961, 6914, 6709, 6593, 6579, 6578, 6573, 6371, 6142, 6064, 6014, 5932, 5929, 5803, 5685, 5670, 5460, 5221, 5144, 5096, 5015, 5012, 4887, 4769, 4754, 4545, 4222, 4086, 4083, 3842, 3827, 3162, 2919, 2904, 2002, 1987, 1096, 1082, 184]


In [307]:
# 倒序删除失败index
for idx in failed_index:
    del all_examples[idx]
print("len(all_examples): {}".format(len(all_examples)))

len(all_examples): 8912


In [308]:
# 保证构造的输入不为空
for item in all_examples:
    assert item['model_input'] != ''

# 4. 计算bertscore

In [65]:
from evaluate import load
bertscore = load("bertscore")

In [206]:
bertscore

EvaluationModule(name: "bert_score", module_type: "metric", features: [{'predictions': Value(dtype='string', id='sequence'), 'references': Sequence(feature=Value(dtype='string', id='sequence'), length=-1, id='references')}, {'predictions': Value(dtype='string', id='sequence'), 'references': Value(dtype='string', id='sequence')}], usage: """
BERTScore Metrics with the hashcode from a source against one or more references.

Args:
    predictions (list of str): Prediction/candidate sentences.
    references (list of str or list of list of str): Reference sentences.
    lang (str): Language of the sentences; required (e.g. 'en').
    model_type (str): Bert specification, default using the suggested
        model for the target language; has to specify at least one of
        `model_type` or `lang`.
    num_layers (int): The layer of representation to use,
        default using the number of layers tuned on WMT16 correlation data.
    verbose (bool): Turn on intermediate status update.
    

In [309]:
from tqdm import tqdm

# 使用tqdm对可迭代对象进行包装，实现进度条可视化
# 计算bertscore
for i in tqdm(range(len(all_examples))):
    bs = bertscore.compute(predictions=[all_examples[i]['raw']], references=[all_examples[i]['attacked']], lang="en")
    all_examples[i]['bert_score_precision'] = bs['precision'][0]
    all_examples[i]['bert_score_recall'] = bs['recall'][0]
    all_examples[i]['bert_score_f1'] = bs['f1'][0]

100%|███████████████████████████████████████████████████████████████████████████████████████| 8912/8912 [05:29<00:00, 27.04it/s]


# 5. 计算use_score

In [75]:
from textattack.constraints.semantics.sentence_encoders import UniversalSentenceEncoder


**************************************************
initial save_path successfully!
save_path: /remote-home/kren/exps/TextAttack/tensors/textfooler_imdb_bert_random_1000sample_20221226.pt
**************************************************



In [76]:
use_encoder = UniversalSentenceEncoder(
    threshold=0.840845057,
    metric="angular",
    compare_against_original=False,
    window_size=15,
    skip_text_shorter_than_window=True,
)

In [153]:
def get_use_sim(raw_input, attacked_input):
    raw_embedding, attacked_embedding = use_encoder.encode(
        [raw_input, attacked_input]
    )
    # raw_embedding: 已由<class 'tensorflow.python.framework.ops.EagerTensor'>转换为numpy
    if not isinstance(raw_embedding, torch.Tensor):
        raw_embedding = torch.tensor(raw_embedding)

    if not isinstance(attacked_embedding, torch.Tensor):
        attacked_embedding = torch.tensor(attacked_embedding)

    # 为了满足torch的输入要求，增加一个batch维
    raw_embedding = torch.unsqueeze(raw_embedding, dim=0)
    attacked_embedding = torch.unsqueeze(attacked_embedding, dim=0)

    # cos_sim
    sim_metric = torch.nn.CosineSimilarity(dim=1)
    return sim_metric(raw_embedding, attacked_embedding)

In [310]:
from tqdm import tqdm

# 计算use_sim
for i in tqdm(range(len(all_examples))):
    use_sim = get_use_sim(all_examples[i]['raw'], all_examples[i]['attacked'])
    all_examples[i]['use_sim'] = use_sim.item()

100%|██████████████████████████████████████████████████████████████████████████████████████| 8912/8912 [01:24<00:00, 104.85it/s]


# 5. 保存为csv文件并进行数据分析

In [102]:
import pandas as pd

In [311]:
meta_df = pd.DataFrame(all_examples)

In [312]:
meta_df

,raw,attacked,epsilon,timestep,cont_sim,disc_sim,difference,model_input,bert_score_precision,bert_score_recall,bert_score_f1,use_sim
0,My girlfriend once brought around The Zombie C...,My girlfriend once brought around The Zombie C...,0.10,1,0.9995,0.9999,0.005,My girlfriend once brought around The Zombie C...,0.997637,0.996182,0.996909,0.995793
1,"I saw this film opening weekend in Australia, ...","I saw this film opening weekend in Australia, ...",0.10,1,0.9942,0.9975,0.005,"I saw this film opening weekend in Australia, ...",0.994779,0.992669,0.993723,0.995721
2,I was excited to view a Cataluña´s film in the...,I was excited to view a Cataluña´s film in the...,0.10,1,0.9945,0.9995,0.01,I was excited to view a Cataluña´s film in the...,0.991487,0.993112,0.992299,0.989435
3,Terrible movie. Nuff Said.<br /><br />These Li...,Terrible movie. Nuff Said.<br /><br />These Li...,0.10,1,0.9992,0.9996,0.005,Terrible movie. Nuff Said.<br /><br />These Li...,0.994131,0.990593,0.992359,0.997162
4,"I saw the capsule comment said ""great acting.""...","I saw the capsule comment said ""great acting.""...",0.10,1,0.9996,0.9996,0.005,"I saw the capsule comment said ""great acting.""...",0.995719,0.994117,0.994917,0.994693
...,...,...,...,...,...,...,...,...,...,...,...,...
8907,"""Bread"" very sharply skewers the conventions o...","""Fuchsia"" very indicator gratifying the franço...",0.57,30,0.7998,0.8008,0.005,"""Bread[Fuchsia]"" very sharply[indicator] skewe...",0.880684,0.827222,0.853116,0.371166
8908,I can't remember many films where a bumbling i...,snort can't rigged erratic circulates where a ...,0.35,30,-0.5608,-0.5748,0.02,I[snort] can't remember[rigged] many[erratic] ...,0.826505,0.792810,0.809307,0.293184
8909,Allen and Julie move into a cabin in the mount...,Allen and Julie eligibility into a wl in the w...,0.54,30,0.9083,0.9034,0.005,Allen and Julie move[eligibility] into a cabin...,0.930775,0.890104,0.909985,0.778371
8910,I definitely recommend reading the book prior ...,I definitely recommend reading the book prior ...,0.26,30,0.9927,0.9879,0.005,I definitely recommend reading the book prior ...,0.953927,0.934702,0.944217,0.932000


In [313]:
# a. imdb-bert-textfooler
csv_folder_path = '/remote-home/kren/exps/FreeLB/hf424_torch182/meta_data/imdb_bert_textfooler'

# b. imdb-bert-random
# csv_folder_path = '/remote-home/kren/exps/FreeLB/hf424_torch182/meta_data/imdb_bert_rand'

csv_file_name = 'preprocessed_data.csv'
csv_save_path = csv_folder_path + '/' + csv_file_name
meta_df.to_csv(csv_save_path)

## a. < 0.005

In [314]:
print(len(meta_df[meta_df['difference'] == '0.005']))

7878


## b. 不同时间段

In [315]:
range_list = [(0, 0.1), (0.1, 0.2), (0.2, 0.3), (0.3, 0.4), (0.4, 0.5), (0.5, 0.6), (0.6, 0.7), (0.7, 0.8)]
for lower, upper in range_list:
    range_num = len(meta_df[meta_df['epsilon'] > lower][meta_df['epsilon'] <= upper])
    print("({},{}]: {}".format(lower, upper, range_num))

(0,0.1]: 4630
(0.1,0.2]: 2265
(0.2,0.3]: 1075
(0.3,0.4]: 528
(0.4,0.5]: 224
(0.5,0.6]: 104
(0.6,0.7]: 54
(0.7,0.8]: 32


/remote-home/kren/anaconda3/envs/hf424_torch182/lib/python3.7/site-packages/ipykernel_launcher.py:3: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  This is separate from the ipykernel package so we can avoid doing imports until


# 6. 划分训练集和测试集(80% train)

In [316]:
len_all_examples = len(all_examples)
len_all_examples

8912

In [317]:
# 9158
eval_idx_list = [idx for idx in range(len_all_examples) if idx % 5 == 0]
train_idx_list = [idx for idx in range(len_all_examples) if idx % 5 != 0]

In [318]:
train_df = meta_df.loc[train_idx_list]
eval_df = meta_df.loc[eval_idx_list]

In [319]:
# a. imdb-bert-textfooler
folder_path = '/remote-home/kren/exps/FreeLB/hf424_torch182/meta_data/imdb_bert_textfooler'

# b. imdb-bert-random
# folder_path = '/remote-home/kren/exps/FreeLB/hf424_torch182/meta_data/imdb_bert_rand'

In [320]:
eval_df.to_csv(folder_path + '/eval_data.csv')
train_df.to_csv(folder_path + '/train_data.csv')

# END